Two important resources for this:
  * [In-depth walkthrough setting up IBMQ account in qiskit, obtaining providers/backends and running jobs](https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
  * [IBMQ backend class, has certain features / commands that differ from simulator backends](https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.IBMQBackend.html#qiskit.providers.ibmq.IBMQBackend)

In [1]:
from qiskit import IBMQ

In [2]:
# token is obtained from IBMQ expierence website, just set up an account at https://quantum-computing.ibm.com/ and once run
# IBMQ.enable_account(token)
# IBMQ.save_account(token)

In [3]:
IBMQ.load_account() # this then automatically loads your saved account

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [4]:
IBMQ.providers() # list of providers, in our case just the standard open one since we dont have any special access (yet!)

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>]

In [5]:
provider = IBMQ.get_provider(hub='ibm-q-research')

### Backends
list and choose a suitable backend

In [6]:
provider.backends(simulator=False, operational=True)

[<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>,
 <IBMQBackend('ibmq_rome') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>,
 <IBMQBackend('ibmq_casablanca') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-res

show the one that has more than 10 qubits and is operational

In [7]:
provider.backends(filters=lambda x: x.configuration().n_qubits >= 10
                                    and not x.configuration().simulator
                                    and x.status().operational==True)

[<IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>]

show least busy device (in terms of jobs in the queue with 5 qubits

In [8]:
from qiskit.providers.ibmq import least_busy

small_devices = provider.backends(filters=lambda x: x.configuration().n_qubits == 5
                                   and not x.configuration().simulator)
least_busy(small_devices)

<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>

In [18]:
backend = least_busy(small_devices)

In [19]:
#backend = provider.backends(simulator=False, operational=True)[7]

In [20]:
backend

<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-research', group='icfo-barcelona-1', project='main')>

In [21]:
status = backend.status()
print("pending jobs: ", status.pending_jobs)
print("maximum jobs to submit: ", backend.job_limit().maximum_jobs)
print("operational: ", status.operational)

pending jobs:  10
maximum jobs to submit:  20
operational:  True


last 5 jobs ran on this backend

In [22]:
ran_job = None
for ran_job in backend.jobs(limit=5):
    print(str(ran_job.job_id()) + " " + str(ran_job.status()))

### Jobs

In [38]:
import qiskit

In [40]:
qr = qiskit.QuantumRegister(3)
cr = qiskit.ClassicalRegister(3)
circuit = qiskit.QuantumCircuit(qr, cr)
circuit.x(qr[0])
circuit.x(qr[1])
circuit.ccx(qr[0], qr[1], qr[2])
circuit.cx(qr[0], qr[1])
circuit.measure(qr, cr)

In [42]:
mapped_circuit = qiskit.compiler.transpile(circuit, backend=backend)
#qobj = qiskit.compiler.assemble(mapped_circuit, backend=backend, shots=1024) # apparently deprecated and or unnecessary
job = backend.run(mapped_circuit)

In [50]:
print(status.pending_jobs)
job.status()

26


<JobStatus.QUEUED: 'job is queued'>

In [51]:
job.job_id()

'60a522f3f9b1c6b392c5038f'

In [43]:
retrieved_job = backend.retrieve_job(job.job_id())

In [52]:
# once job is done this should yield result
#result = retrieved_job.result()
#counts = result.get_counts()
#print(counts)

### Job Manager

In [54]:
from qiskit.providers.ibmq.managed import IBMQJobManager

sim_backend = provider.get_backend('ibmq_qasm_simulator')
circs = qiskit.compiler.transpile([circuit]*20, backend=sim_backend)

# Submit them all to the backend
job_manager = IBMQJobManager()
job_set = job_manager.run(circs, backend=sim_backend, name='foo')

In [55]:
print(job_set.report())

Job set name: foo
          ID: 2c9eb6b6c2c54dd2adbb543d17efdaef-16214354877853491
        tags: []
Summary report:
       Total jobs: 1
  Successful jobs: 1
      Failed jobs: 0
   Cancelled jobs: 0
     Running jobs: 0
     Pending jobs: 0

Detail report:
  experiments: 0-19
    job index: 0
    job ID: 60a524601a71edc100ff8cbd
    name: foo_0_
    status: job has successfully run
